# NLP Analysis : Topic analysis et grammatical parsing

Objectif : produire une analyse par topic --> résultats non concluants, aucun réel pattern ou info pertinente ne semble pouvoir être extraite de ces topics. Le texte des tweets est à priori trop court et les tweets se ressemblent trop pour que des topics émergent. En effet, les tweets traitent de la question des moyens de paiement, une thématique déà très précise. 

In [1]:
from functools import reduce
import random
import re
import time
from gensim.utils import deaccent
import numpy as np
import pandas as pd
import tqdm
import matplotlib.pyplot as plt
pd.options.display.max_colwidth = None
pd.options.display.max_columns = 30
#from wordcloud import WordCloud
import wordcloud
from nltk.corpus import stopwords
import seaborn as sns
import os
import _pickle as cPickle

#pour topic analysis 
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
#fonctions utilitaires 

def query_to_words(query):
    query = str.lower(query)
    query = query.replace("(", "").replace(")", "")
    query = query.replace('"', '')
    splitted = re.split(pattern=" or | and ", string=query)
    splitted = [word.strip() for word in splitted]
    return splitted

def remove_stopwords(texts, stopwords):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords] for doc in texts]

def make_bigrams(texts):
    bigram = gensim.models.Phrases(data_words, min_count=3, threshold=5) # higher threshold fewer phrases.
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return [bigram_mod[doc] for doc in texts]

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
        

def lemmatization(nlp, texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

def query_to_words(query):
    query = query.replace("(", "").replace(")", "")
    query = query.replace('"', '')
    splitted = re.split(pattern="OR|AND", string=query)
    splitted = [word.strip() for word in splitted]
    return splitted

def get_dep_info(tweet, obj_only=False):
    lst = []
    doc = nlp(tweet)
    
    for token in (token for token in doc if token.text in WORDS_QUERY):
        if token.dep_ == 'obj':
            lst += [{"word": token.text, "parent": token.head.text, 'enfant': [child.text for child in token.children]}]
    
    return lst

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
#Import data
df_cash = pd.read_pickle(f"/home/cash/output/cleaned_queries/with_geoloc/query_fr_cash_cleaned_geoloc.pickle")
df_cash["datetime"] = pd.to_datetime(df_cash["datetime"])
df_cash["created_at"] = df_cash["datetime"]
df_cash.set_index("datetime", inplace=True)

df_cb = pd.read_pickle(f"/home/cash/output/cleaned_queries/with_geoloc/query_fr_cb_cleaned_geoloc.pickle")
df_cb["datetime"] = pd.to_datetime(df_cb["datetime"])
df_cb["created_at"] = df_cb["datetime"]
df_cb.set_index("datetime", inplace=True)

df =  pd.concat([df_cash, df_cb], ignore_index = True)
print(len(df))
df = df.drop_duplicates()

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


174540


### 1. Topic analysis

In [4]:
# Importe et complète la liste de stopwords français
stop_words = set(stopwords.words('french'))
more_french_stopwords = [
    "plus", "donc", "juste", "aussi", "toujours", "car", "rien", "aucun", "encore", "déjà", "peu", "enfin", "ni", 
    "parce", "depuis", "jamais", "après", "autre", "non","oui", "si", "trop", "là", "avant", "comme","chez", "quand", "personne", 
    "ça", "ca","meme", "memes","mem", "quoi", "ou", "où","comment","moins","tre","trop","alors","donc", "lors", "coup", 
    "ici","amp","bientot", "bientôt",
    "faire","fais", "fait", "faut","va", "avoir", "dit", "dire", "peut", "peux","etre", "être",
    "cette", "ceux","cettes","cela", "leurs"
]
stop_words.update(more_french_stopwords)

# Importe un modèle LDA
nlp = spacy.load("fr_core_news_sm")

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [7]:
# Pré-traite le texte pour appliquer LDA

data_words = df.clean_tweet.values.tolist()

if (os.path.isfile("/home/cash/output/topic_analysis/data_lemmatized_df_fr.pickle")==False): 
    
    data_words_liste = list(sent_to_words(data_words))
    data_words_nostops = remove_stopwords(data_words_liste, stop_words)
    data_words_bigrams = make_bigrams(data_words_nostops)
    data_lemmatized = lemmatization(nlp,data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    
    with open(r"/home/cash/output/topic_analysis/data_lemmatized_df_fr.pickle", "wb") as output_file:
        cPickle.dump(data_lemmatized, output_file)
    
else : 
    with open(r"/home/cash/output/topic_analysis/data_lemmatized_df_fr.pickle", "rb") as input_file:
        data_lemmatized = cPickle.load(input_file)


/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
# Create Dictionary
#id2word = corpora.Dictionary(data_words_bigrams)
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
#texts = data_words_bigrams
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus[:1])

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 4), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1)]]


In [9]:
# Build LDA model
if (os.path.isfile('/home/cash/output/topic_analysis/lda_8topics_df_fr.model')==False): 

    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                                id2word=id2word,
                                                num_topics=8, 
                                                random_state=100,
                                                update_every=1,
                                                chunksize=100,
                                                passes=10,
                                                alpha='auto',
                                                per_word_topics=True)
                   
    lda_model.save('/home/cash/output/topic_analysis/lda_8topics_df_fr.model')
    
else : 
    lda_model =  gensim.models.LdaModel.load('/home/cash/output/topic_analysis/lda_8topics_df_fr.model')

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Visualiser ce que donne le modèle LDA

In [10]:
# Décrire les topics obtenus avec les mots les plus fréquents
pprint(lda_model.print_topics(num_words=20)) #num_topics=20, 
doc_lda = lda_model[corpus]

[(0,
  '0.057*"compte" + 0.047*"euro" + 0.030*"merci" + 0.024*"vouloir" + '
  '0.024*"an" + 0.023*"retirer" + 0.019*"banqu" + 0.015*"virement" + '
  '0.015*"point" + 0.015*"depenser" + 0.014*"mai" + 0.013*"cadeau" + '
  '0.013*"pourquoi" + 0.012*"apple" + 0.011*"marche" + 0.011*"dernier" + '
  '0.011*"fnac" + 0.010*"chose" + 0.009*"fintech" + 0.009*"caisse"'),
 (1,
  '0.052*"contact" + 0.045*"mobile" + 0.036*"cb" + 0.031*"tout" + '
  '0.022*"passer" + 0.022*"mois" + 0.019*"argent" + 0.018*"devoir" + '
  '0.015*"client" + 0.013*"premier" + 0.012*"code" + 0.011*"banque" + '
  '0.011*"million" + 0.011*"tre" + 0.010*"paypal" + 0.009*"faire" + '
  '0.009*"attendre" + 0.008*"journe" + 0.006*"autre" + 0.006*"covid"'),
 (2,
  '0.081*"depense" + 0.064*"payer" + 0.022*"bien" + 0.022*"bleu" + '
  '0.021*"aller" + 0.017*"apre" + 0.017*"fois" + 0.017*"eter" + 0.017*"voir" + '
  '0.014*"prix" + 0.013*"mettre" + 0.010*"demande" + 0.009*"jusqu" + '
  '0.009*"semaine" + 0.008*"credit" + 0.008*"temps" +

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [11]:
# Identifier les tweets selon leur topic
from operator import itemgetter
res= pd.DataFrame(columns=['text','topic','proba'])
seuil = 0.2
for index in range(4998) :
    proba=lda_model.get_document_topics(corpus[index])
    text = data_words[index]
    res_interm = [topic for topic in proba if topic[1]>=seuil]
    if not res_interm:
        a="empty"
    else : 
        res = res.append(
            {"text": text,
             "topic": max(res_interm,key=itemgetter(1))[0], 
             "proba": max(res_interm,key=itemgetter(1))[1]
            }, ignore_index=True)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [12]:
# Cela permet de voir les tweets qui correspondent le plus au topic 6 : on ne voit pas trop ce qu'ils ont en commun. 
# Penser à ajuster le seuil ci-dessus. 
res= res.sort_values("topic")
res[res.topic == 6]

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,text,topic,proba
4838,@braindegeek et certains restaurants à paris non pas le bon appareil pour accepter les visa étrangères alors ils demandent des espèces (comprendre retirer dans un guichet) ça va faire mal la psd2 empagnie pour eux,6,0.274842
4922,"je rétorque ""nan je n ai que ma carte sur moi "" il me regarde sévère et dans le plus grand des calmes me sort ""dis moi crois tu que ceci est une excuse pour ne pas donner de l argent ? il y a des distributeur ici tu peut allez me retirer des billets vas y !"" whaaaat?? ///1acfykis4p",6,0.220253
4710,👀 une permanente de cheveux très mathématiques ! pour débuter notre série #afficheshistoriquesbp voici cette affiche des années 70 faisant la promotion d intercarte @banquepopulaire 💳 la première carte de paiement et de retrait à l international ! ///3keqtku0yh,6,0.229621
3557,"la relation client en boutique c est + ce que c était avant ""bonjour monsieur roybet "" actuellement ""c est à qui le tour ? vous avez la carte de fidélité ? paiement sans contact ? on prend pas les tickets resto / on rend pas la monnaie ! #relationclient #shop",6,0.245783
3499,@rmvrdr je sais pas si il n y a que moi mais quand je paye en carte ou autre j ai pas l impression de payer le cash te permet aussi d avoir une limite physique boudgetaire j ai un billet de 50€ pour les courses par exemple,6,0.250575
...,...,...,...
2204,cartes bancaires chèques espèces quels moyens de paiement de vos clients êtes-vous obligés d accepter ? |… ///osyk41xwss,6,0.334970
2210,🤩🤩désormais sur ///wpsmixldjf vous pouvez régler en espèces par ticket resto chèques vacances et chèque kdo mon shopping calais 😉 #hovercraft #lovewelsh #livraison #clickandcollect ///jznp43pomu,6,0.305429
2211,hier soir je vais au bar et le serveur nous dit que c est paiement uniquement en espèce pcq son tpe est en panne et pendant la soirée une copine se fait voler son sac c est pas il nous dit « bon avec toutes vos galères je peux vous laisser payer en cb » et il sort son tpe,6,0.259951
2216,aujourd hui on peut payer à peu près tout par carte via sans contact ou non y compris les transports en commun alors va-t-on vers la fin des #espèces? ///pxlznw3m0o ///ufe1l0prca,6,0.267393


In [13]:
# Calculer des métriques pour connaître la performance du modèle. 
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -9.887163611737485

Coherence Score:  0.257923618231118


In [14]:
# Outil pour visualiser les topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.8/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.163744 -0.112838       1        1  22.786612
6     -0.237274  0.283587       2        1  21.388316
2     -0.176908 -0.311534       3        1  20.121168
1     -0.069865  0.128143       4        1  15.754347
0      0.104598 -0.004829       5        1   9.552477
4      0.179137  0.008342       6        1   4.649572
7      0.182219  0.004885       7        1   3.095022
3      0.181836  0.004244       8        1   2.652486, topic_info=            Term           Freq          Total Category  logprob  loglift
4          carte  111123.000000  111123.000000  Default  30.0000  30.0000
10      paiement   51173.000000   51173.000000  Default  29.0000  29.0000
401      depense   34961.000000   34961.000000  Default  28.0000  28.0000
68         payer   27290.000000   27290.000000  Default  27.0000  27.0000
117      acheter   24978.000000   24978.000000  Default  26.0000  26.0000
...          ...            ...            ...      ...      ...      ...
1296     laisser     499.757847     500.748055   Topic8  -4.7304   3.6277
4696       video     493.284599     494.274729   Topic8  -4.7434   3.6277
657          ami     477.755715     478.745925   Topic8  -4.7754   3.6276
262     commerce     470.708890     471.698631   Topic8  -4.7903   3.6276
862   simplement     431.252067     432.242576   Topic8  -4.8778   3.6274

[274 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
507       4  0.999607  abonnement
2159      6  0.998578        acce
13        2  0.999193     accepte
1107      8  0.998191      accord
27        1  0.999931       achat
...     ...       ...         ...
1565      8  0.999245         vol
1951      6  0.998459        vole
345       5  0.999799     vouloir
1980      1  0.999676    vraiment
3281      7  0.998330     youtube

[250 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 7, 3, 2, 1, 5, 8, 4])